In [ ]:
using Pkg
Pkg.develop(path="../../Tambo/")
using Tambo
Pkg.activate(".")

In [ ]:
Pkg.add("CurveFit")

In [ ]:
using JLD2
using Glob

# Extrapolate out computed rates to get scaling
using Optim
using CurveFit

# Compute effective area
using FHist
using StatsBase

# interpolate and integrate effective areas
using SmoothingSplines
using QuadGK

# Plotting
using LaTeXStrings
using Makie
using CairoMakie

# QoL
using ProgressBars
include("../../scripts/2_trigger_events/trigger_defs.jl")

In [ ]:
const year = 365.25 * 24 * 3600 * units.second

In [ ]:
# jldf = jldopen("/Users/jlazar/Downloads/larger_valley_hard_00000_00001_150_1000.jld2")

In [ ]:
sim_example = jldopen("../../ack_scratch/sims/hard_spectrum/larger_valley_00000_00001.jld2")
sim_config = Dict(kv[1]=>kv[2] for kv in sim_example["config"])
# sim_config["geo_spline_path"] = "../resources/tambo_spline.jld2"

injection_config = Dict(
    "thetamin" => rad2deg(sim_config[:θmin]),
    "thetamax" => rad2deg(sim_config[:θmax]),
    "phimin" => rad2deg(sim_config[:ϕmin]),
    "phimax" => rad2deg(sim_config[:ϕmax]),
    "emin" => sim_config[:emin] / units.GeV,
    "emax" => sim_config[:emax] / units.GeV,
    "gamma" => sim_config[:γ],
    "r_injection" => sim_config[:r_injection] / units.m,
    "l_endcap" => sim_config[:l_endcap] / units.m,
    "xs_dir" => "../../resources/cross_sections/tables/",
    "xs_model" => "csms",
    "nu_pdg" => sim_config[:ν_pdg],
    "interaction" => "CC",
)

geo_config = Dict(
    "geo_spline_path" => "../../resources/tambo_spline.jld2",
    "tambo_coordinates" => rad2deg.([sim_config[:tambo_coordinates].latitude, sim_config[:tambo_coordinates].longitude]),
    "plane_orientation" => sim_example["config"][:plane_orientation].proj
)

geo = Tambo.Geometry(geo_config)
plane = Tambo.Plane(normal_vecs.larger_valley, Tambo.coords.larger_valley, geo)
injector = Tambo.Injector(injection_config, geo)

close(sim_example)

In [ ]:
edges = 10 .^ LinRange(-0.3, 3, 15) .* units.PeV
cents = (edges[1:end-1] .+ edges[2:end]) ./2
widths = edges[2:end] .- edges[1:end-1]

array_lengths = [200, 300, 400, 500, 600, 750, 800, 900, 1000, 1500, 2000, 2500, 3000, 3500]

nmodules = Int[]
x = ones((length(array_lengths), length(cents), 2))

for (idx, ℓ) in ProgressBar(enumerate(array_lengths))
    
    fs = glob("larger_valley_hard_00000_000??_150_$(ℓ).jld2", "/Users/jlazar/Downloads/")
    
    triggered_events = Tambo.InjectionEvent[]
    nmodule = nothing
    nevent = 0
    
    for f in fs
        
        jldf = jldopen(f)
        
        if typeof(nmodule)==Nothing
            nmodule = length(Tambo.make_detector_array(
                jldf["array_config"]["length"],
                jldf["array_config"]["deltas"],
                jldf["array_config"]["altmin"],
                jldf["array_config"]["altmax"],
                plane,
                geo,
                Tambo.SVector{3}(1,2,1)
            ))
        end
        subsimset = split(f, "_")[end-2]
        sim = jldopen("../../ack_scratch/sims/larger_valley_00000_$(subsimset).jld2")

        nevent += sim["config"][:n]

        hit_maps = jldf["hit_map"]
        triggered_idxs = Int[]
        for k in keys(hit_maps)
            v = hit_maps[k]
            if did_trigger(v)
                push!(triggered_idxs, parse(Int, k))
            end

        end
        triggered_events = vcat(triggered_events, sim["injected_events"][triggered_idxs])
    end
    es = map(x->x.initial_state.energy, triggered_events)
    oneweights = Tambo.oneweight.(triggered_events, Ref(injector), Ref(injector.xs)) ./ nevent
    wgts = Weights(oneweights)
    h = Hist1D(es, wgts, edges)
    x[idx, :, 1] = h.hist.weights ./ widths
    # TODO Compute the δ
    push!(nmodules, nmodule)
end

In [ ]:
effectivearea_Ω = Float64[]

for idx in 1:size(x)[1]
    fig = Figure()
    ax = Axis(fig[1,1])
    stairs!(ax, nmodules, x[:, idx, 1] ./ units.cm^2, step=:center)
    
    b, m = linear_fit(nmodules[1:end-3], x[1:end-3, idx, 1])
    lines!(ax, nmodules, (m .* nmodules .+ b) ./ units.cm^2)
    display(fig)
    push!(effectivearea_Ω, m * 5000 + b)
end

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xscale=log10, yscale=log10)
stairs!(ax, cents, effectivearea_Ω ./ units.cm^2, step=:center)

fig

In [ ]:
function g(p, n, y, δy)
    m, b = p
    yp = m .* n .+ b
    return sum((yp .- y) .^ 2 ./ δy .^ 2)
end

In [ ]:
spl = SmoothingSplines.fit(
    SmoothingSpline,
    log.(cents),
    log.(effectivearea_Ω),
    0.5
)

fine_es = (10 .^ LinRange(log(10, 0.3), 3, 100)) .* units.PeV
interped_effa = exp.(predict.(Ref(spl), log.(fine_es)))

fig = Figure(fontsize=20)

ax = Axis(
    fig[1, 1],
    xscale=log10,
    yscale=log10,
    xlabel=L"E_{\nu}~\left[\mathrm{GeV}\right]",
    ylabel=L"\Omega\, A_{\mathrm{eff}}~\left[\mathrm{m}^{2}\,\mathrm{sr}\right]"
)

stairs!(
    ax,
    cents ./ units.GeV,
    effectivearea_Ω ./ units.m^2,
    step=:center
)

lines!(ax, fine_es ./ units.GeV, interped_effa / units.m^2, fine_es)

fig

In [ ]:
pavel_norms = [1.8995052121226763e-19, 6.269206947352916e-21, 3.2297181355164026e-22, 1.9772329206625328e-23, 1.6327753266356979e-24, 1.9077918937306844e-25, 3.064665871092821e-26, 9.073827181711629e-27]
pavel_cents = 10 .^ (14.5:0.5:18)

In [ ]:
γ = 2
norm0 = 1.0e-1
norms = Float64[]

for ecent in pavel_cents
    emin = ecent / 10^0.5
    emax = ecent * 10^0.5
    base_norm = norm0 / units.GeV / units.cm^2 / units.second * (1 /(ecent))^-γ
    
    Φ = Tambo.PowerLaw(γ, 1000units.GeV, 1e9units.GeV, base_norm)
    f(e) = Φ(e) * exp(predict(spl, log(e))) * 10 * year
    g(le) = exp(le) * f(exp(le))

    I, err = quadgk(g, log(emin), log(emax))
    if err / I > 1e-5
        error()
    end
    push!(norms, 2.44 * norm0 / I)
end

fig = Figure(fontsize=20)

ax = Axis(
    fig[1, 1],
    xscale=log10,
    yscale=log10,
    limits=(3e5, 1e9, 1e-10, 1e-5),
    xlabel=L"E_{\nu}~\left[\mathrm{GeV}\right]",
    ylabel=L"E_{\nu}^{2}\,\Phi_{2\nu}~\left[\mathrm{GeV}\,\mathrm{cm}^{-2}\,\mathrm{s}^{-1}\,\mathrm{sr}^{-1}\right]"
)

lines!(ax, pavel_cents / units.GeV, norms .* (pavel_cents ./ units.GeV).^2)
lines!(ax, pavel_cents / units.GeV, pavel_norms .* (pavel_cents ./ units.GeV).^2)

display(fig)